#### 目標：取得當周電影院的票房資料，每周爬一次
資料來源：《全國電影票房統計資訊》https://boxofficetw.tfai.org.tw/


In [ ]:
##### 安裝套件 #####
import os
import requests
import pandas as pd
import json
from datetime import datetime, timedelta

print("安裝成功")


安裝成功


In [18]:
##### 常數&參數設定 #####
import os

# 儲放位置
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '../../../..'))
RAW_PATH_JSON = os.path.join(PROJECT_ROOT, 'data', 'raw', 'boxoffice', 'json')
os.makedirs(RAW_PATH_JSON, exist_ok=True)
# PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), '../../../..'))

# RAW_PATH_JSON = os.path.join(PROJECT_ROOT, 'data', 'raw', 'boxoffice', 'json')

# os.makedirs(RAW_PATH_JSON, exist_ok=True)

# RAW_PATH_JSON = "./../../data/raw/boxoffice/json/"
# RAW_PATH_CSV = "data/raw/boxoffice/csv/"

# 政府票房統計API（全國電影票房統計）
BASE_URL = "https://boxofficetw.tfai.org.tw/stat/qsl?mode=Week&start=&ascending=false&orderedColumn=ReleaseDate&page=0&size=&region=all"  

In [16]:
import os
import requests
import json
from datetime import datetime, timedelta
from common.date_utils import get_last_week_range


# RAW_PATH_JSON = "./../../data/raw/boxoffice/json/"

# os.makedirs(RAW_PATH_JSON, exist_ok=True)


# 取得資料
def fetch_boxoffice_json(date_str: str):
    print('RAW_PATH_JSON',RAW_PATH_JSON)
    """
    從官方 API 下載指定週的票房資料(JSON) 並存檔
    """

    # 取得上週起訖日期
    date_range = get_last_week_range()
    print(date_range)

    url = "https://boxofficetw.tfai.org.tw/stat/qsl"
    params = {
        "mode": "Week",
        "start": date_range['startDate'],
        "ascending": "false",
        "orderedColumn": "ReleaseDate",
        "page": 0,
        "size": "",  # 留空以抓全部
        "region": "all"
    }

    print("params",params)
    response = requests.get(url, params=params)
    response.encoding="utf-8"

    data = response.json()
    print("data",data)

    # 檔名
    week_label = f"{datetime.now().strftime('%Y')}W{datetime.now().strftime('%U')}"
    file_path = os.path.join(RAW_PATH_JSON, f"boxoffice_{week_label}.json")

    # 儲存原始 JSON
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ 已儲存原始資料：{file_path}")
    # return data







if __name__ == "__main__":
    fetch_boxoffice_json("2025-10-08")


RAW_PATH_JSON ./../../data/raw/boxoffice/json/
{'startDate': '2025-09-29', 'endDate': '2025-10-05'}
params {'mode': 'Week', 'start': '2025-09-29', 'ascending': 'false', 'orderedColumn': 'ReleaseDate', 'page': 0, 'size': '', 'region': 'all'}
data {'data': {'page': 0, 'size': 92, 'pageCount': 1, 'dataItemCount': 92, 'fallback': False, 'start': '2025-09-29T00:00:00', 'end': '2025-10-05T00:00:00', 'dataItems': [{'movieId': '31807', 'rank': 1, 'region': '中華民國', 'name': '泥娃娃', 'releaseDate': '2025-09-28', 'publisher': '文化內容策進院##合影視有限合夥##必應創造股份有限公司##能率影業股份有限公司##阿榮影業股份有限公司##天予電影有限公司', 'dayCount': 2, 'theaterCount': 1, 'amount': 147490.0, 'amountRate': 1.55526680526681, 'tickets': 601, 'ticketRate': 1.7072072072072073, 'marketShare': 0.7302551640340219, 'totalDayCount': 2, 'totalAmount': 205210.0, 'totalTickets': 823, 'firstWeekAmount': 0.0, 'totalWeekendAmount': 57720.0}, {'movieId': '31799', 'rank': 2, 'region': '日本', 'name': '電影版 神奇柑仔店 錢天堂', 'releaseDate': '2025-09-26', 'publisher': 'TOHO CO

In [7]:
# 取得上週一到日的起訖日期
def get_last_week_range(reference_date: datetime = None):
    """
    取得上週一到日的起訖日期
    回傳：
    {
        "startDate": "YYYY-MM-DD",
        "endDate":   "YYYY-MM-DD"
    }
    """
    if reference_date is None:
        today = datetime.today()
    else:
        today = reference_date

    # 計算這週的週一
    this_monday = today - timedelta(days=today.weekday())
    # 上週一
    last_monday = this_monday - timedelta(weeks=1)
    # 上週日
    last_sunday = last_monday + timedelta(days=6)

    return {
        "startDate": f"{last_monday.year}-{str(last_monday.month).zfill(2)}-{str(last_monday.day).zfill(2)}",
        "endDate": f"{last_sunday.year}-{str(last_sunday.month).zfill(2)}-{str(last_sunday.day).zfill(2)}"
    }

# 測試例
print(get_last_week_range())
# 結果：{'startDate': '2025-09-29', 'endDate': '2025-10-05'}

{'startDate': '2025-09-29', 'endDate': '2025-10-05'}


In [ ]:
# src/movie_recommendation/crawler/boxoffice_api.py

import os
import requests
import json
from datetime import datetime


os.makedirs(RAW_PATH_JSON, exist_ok=True)

def fetch_boxoffice_json(date_str: str):
    """
    從官方 API 下載指定週的票房資料(JSON) 並存檔
    """
    url = "https://boxofficetw.tfai.org.tw/stat/qsl"
    params = {
        "mode": "Week",
        "start": date_str,
        "ascending": "false",
        "orderedColumn": "ReleaseDate",
        "page": 0,
        "size": "",  # 留空以抓全部
        "region": "all"
    }

    response = requests.get(url, params=params)
    response.raise_for_status()

    data = response.json()

    # 檔名
    week_label = f"{datetime.now().strftime('%Y')}W{datetime.now().strftime('%U')}"
    file_path = os.path.join(RAW_PATH_JSON, f"boxoffice_{week_label}.json")

    # 儲存原始 JSON
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"✅ 已儲存原始資料：{file_path}")
    return data


if __name__ == "__main__":
    fetch_boxoffice_json("2025-10-08")
